In [ ]:
!pip -q install jupyter-dash
!pip -q install --upgrade plotly
!pip -q install dash_bootstrap_components

!pip -q install ibmcloudant>=0.0.33
!pip -q install ibm-cloud-sdk-core>=3.9.0
!pip -q install ibm-cos-sdk>=2.10.0

     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 1.0MB 32.0MB/s 
     |████████████████████████████████| 3.5MB 37.1MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 1.8MB 37.6MB/s 
     |████████████████████████████████| 358kB 42.3MB/s 
     |████████████████████████████████| 19.1MB 1.5MB/s 
     |████████████████████████████████| 194kB 6.4MB/s 


In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

import pandas as pd
import os
import json
from ibmcloudant.cloudant_v1 import CloudantV1
from ibm_botocore.client import Config
import ibm_boto3
from io import BytesIO

In [ ]:
TEMPLATE = 'plotly_white'
COLORMAP = px.colors.diverging.balance

### Base de datos original

In [ ]:
COS_ENDPOINT = "https://s3.fra.eu.cloud-object-storage.appdomain.cloud"
COS_API_KEY_ID = "3yiHID6Tkcj0xtCywqU18UVfrEeraqsAKLwOm3O94YRN"
COS_INSTANCE_CRN = "fef11c7d-529e-4e45-9daf-e7f424189277"

os.environ["MODEL_CATALOG_TFM_URL"] = 'https://108eeb7e-1bfd-4caa-b305-9f0e89fa4cae-bluemix.cloudantnosqldb.appdomain.cloud'
os.environ["MODEL_CATALOG_TFM_APIKEY"] = 'lky1auqtTJzVWseuP41A0MCQvUHKmTaycGEK808kh0pR'

In [ ]:
cos_client = ibm_boto3.client("s3",
        ibm_api_key_id=COS_API_KEY_ID,
        ibm_service_instance_id=COS_INSTANCE_CRN,
        config=Config(signature_version="oauth"),
        endpoint_url=COS_ENDPOINT
    )

In [ ]:
client = CloudantV1.new_instance(service_name="MODEL_CATALOG_TFM")
db_name = "predictions"

response_predictions = client.post_all_docs(
  db=db_name,
  include_docs=True
).get_result()

In [ ]:
ds_job = cos_client.get_object(Bucket='cos-ia-online-bbn-group04-data', Key='ds_job.csv')
DATA_BYTES = BytesIO(ds_job['Body'].read())

df = pd.read_csv(DATA_BYTES, sep=",")
df.head()

,empleado_id,ciudad,indice_desarrollo_ciudad,genero,experiencia_relevante,universidad_matriculado,nivel_educacion,educacion,experiencia,tamano_compania,tipo_compania,ultimo_nuevo_trabajo,horas_formacion,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [ ]:
for res_row in response_predictions['rows']:
  dic_row = dict(res_row['doc'])
  del dic_row['_id']
  del dic_row['_rev']
  del dic_row['model_version']

  if dic_row['empleado_id'] not in df.empleado_id.values:
    print(dic_row)
    df = df.append(dic_row, ignore_index=True)

{'ciudad': 'Sabadell', 'indice_desarrollo_ciudad': 0.0, 'genero': 'Male', 'experiencia_relevante': 'Has relevent experience', 'universidad_matriculado': 'no_enrollment', 'nivel_educacion': 'Graduate', 'educacion': 'STEM', 'experiencia': '<1', 'tamano_compania': '<10', 'tipo_compania': 'Pvt Ltd', 'ultimo_nuevo_trabajo': '1', 'horas_formacion': 10, 'target': 1.0, 'empleado_id': 123}
{'ciudad': 'Barcelona', 'indice_desarrollo_ciudad': 0.0, 'genero': 'Male', 'experiencia_relevante': 'Has relevent experience', 'universidad_matriculado': 'no_enrollment', 'nivel_educacion': 'Graduate', 'educacion': 'STEM', 'experiencia': '<1', 'tamano_compania': '<10', 'tipo_compania': 'Pvt Ltd', 'ultimo_nuevo_trabajo': '1', 'horas_formacion': 1, 'target': 1.0, 'empleado_id': 1000999}


In [ ]:
df.set_index('empleado_id', inplace=True)

In [ ]:
df.experiencia.replace(">20", '21', inplace=True)
df.experiencia.replace("<1", '0', inplace=True)
df["experiencia"] = pd.to_numeric(df["experiencia"])
df.experiencia.fillna(df.experiencia.mean(), inplace=True)

In [ ]:
df.universidad_matriculado.fillna('No definido', inplace=True)
df.nivel_educacion.fillna('No definido', inplace=True)
df.educacion.fillna('No definido', inplace=True)
df.tipo_compania.fillna('No definido', inplace=True)
df.genero.fillna('Female', inplace=True)

### Funciones del *Dashboard*

In [ ]:
def drawFigure(id_fig, fig):
    return  html.Div([
        dbc.Card(
            dbc.CardBody([         
                dcc.Graph(
                    id=id_fig,
                    figure=fig,
                    config={
                        'displayModeBar': False
                    }
                ) 
            ])
        ),  
    ])

In [ ]:
def drawTextH1(text):
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H1(children=text, className="text-white"),
                ] , style={'textAlign': 'center'}) 
            ]), className="card text-white bg-primary  mb-3"
        ),
    ])

def drawTextH3(text):
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H3(text, className="text-white"),
                ], style={'textAlign': 'center'}) 
            ]), className="card text-white bg-dark  mb-3"
        ),
    ])

def drawTextP(text):
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.P(text),
                ], style={'textAlign': 'left'}) 
            ])
        ),
    ])

### Creación estructura del *Dashboard*

In [ ]:
# Build App
app = JupyterDash(external_stylesheets=[dbc.themes.LUX])

# Sección global
app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            # Sección superior
            dbc.Row([
              dbc.Col(drawTextH1("Dashboard - TFM"))   
            ]),
            html.Br(),
            dbc.Row([
                     dbc.Col([drawTextH3('Cantidad de ejemplos según nivel de educación')]),
                     dbc.Col([drawTextH3('Género')])    
            ]),
            dbc.Row([
                     dbc.Col([drawTextP('''En esta gráfica se muestra la cantidad de personas que contempla el dataset según el nivel de educación. ''')]),
                     dbc.Col([drawTextP('''Distribución según el género de los ejemplos del conjunto de datos.''')])    
            ]),
            html.Br(),
            dbc.Row([
                # Subseccion 1 - Seccion Principal
                dbc.Col([
                    drawFigure('graph-1', px.pie(df, values=df.nivel_educacion.value_counts(dropna=False), names=df.nivel_educacion.unique(), template=TEMPLATE, color_discrete_sequence=COLORMAP))
                ]),

                # Subseccion 2 - Seccion Principal
                dbc.Col([
                    drawFigure('graph-2', px.bar(df, y=df.genero.value_counts(dropna=False), x=df.genero.unique(), color=df.genero.unique(), template=TEMPLATE, color_discrete_sequence=COLORMAP))
                ]),
            ], align='center'), 
			
            html.Br(),

            # Sección intermedia
            dbc.Row([
                     dbc.Col([drawTextH3('Distribución jerarquizada de la educación')]),
                     dbc.Col([drawTextH3('Media de la experiencia según el tipo de educación')])      
            ]),
            dbc.Row([
                     dbc.Col([drawTextP('''En esta sección se muestra un gráfico de sunburst. En este tipo de gráficos se muestran de forma jerarquizada un conjunto de variables determinada.
                     En este caso, se muestran si la persona está matriculado, a que nivel educativo y en que tipo de rama se encuentra. Además, para cada grupo se muestran las horas totales de formación. 
                     Esto permite comparar entre disciplinas distintas y la cantidad de tiempo invertido en adquirir un mayor conocimiento. ''')]),
                     dbc.Col([drawTextP('''Por otro lado, en esta gráfica se muestra la media de experiencia para cada disciplina de conocimiento. ''')]),     
            ]),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    drawFigure('graph-3', px.sunburst(df, path=['universidad_matriculado','nivel_educacion', 'educacion'], values='horas_formacion', template=TEMPLATE, color_discrete_sequence=COLORMAP))
                ]),
                dbc.Col([
                    drawFigure('graph-4', px.bar(df, y=df.groupby('educacion').experiencia.mean(), x=df.educacion.unique(), color=df.educacion.unique(), template=TEMPLATE, color_discrete_sequence=COLORMAP))
                ])
            ], align='center'), 
			
            html.Br(),

            # Sección inferior
            dbc.Row([
                     dbc.Col([drawTextH3('Distribución de la variable objetivo')])    
            ]),
            dbc.Row([
                     dbc.Col([drawTextP('''En esta gráfica se muestra simplemente un conteo de la cantidad de ejemplos disponibles para cada valor de la variable dependiente. ''')])    
            ]),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    drawFigure('graph-5', px.pie(df, values=df.target.value_counts(dropna=False), names=df.target.unique(), template=TEMPLATE, color_discrete_sequence=COLORMAP))
                ])
            ], align='center'),
			
        ]), color = 'light'
    )
])

### Ejecución del *Dashboard*

In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='external')
app.run_server(mode='inline')

Dash app running on:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>